In [7]:
#Analysis for a house flip imports, sort by median household income by zip codes, return URL
import pandas as pd

zillow_props = pd.read_excel("./Scraped Data/Zillow/Properties_Zillow_Aug2.xlsx")
mass_data = pd.read_excel("Massachusetts_IncomeByZipDemographics.xlsx")

def fix_zip(series):
      return series.astype(str).str.extract('(\d+)', expand=False).str.zfill(5)

mass_data['zip_code'] = fix_zip(mass_data['zip_code'])
del mass_data['geoid']
del mass_data['state_name']


In [8]:
#Checking if the datasets are good

zillow_props['zip'] = zillow_props['address'].str.extract(r'(\d{5}\-?\d{0,4})')
del zillow_props['rank']
del zillow_props['property_id']
del zillow_props['latitude']
del zillow_props['longitude']
del zillow_props['currency']
del zillow_props['sold_date']
del zillow_props['is_zillow_owned']
del zillow_props['image']
del zillow_props['listing_type']
del zillow_props['broker_name']
del zillow_props['input']
del zillow_props['listing_url']
zillow_props.head()


,address,price,bathrooms,bedrooms,area,zestimate,rent_zestimate,days_on_zillow,property_url,zip
0,"404 Lincoln St, Lexington, MA 02421",750000.0,2.0,3.0,1848.0 sqft,828200.0,3349.0,75,https://www.zillow.com/homedetails/404-Lincoln...,02421
1,"239-241 Delhi St, Mattapan, MA 02126",750000.0,2.0,6.0,2178.0 sqft,776200.0,3599.0,5,https://www.zillow.com/homedetails/239-241-Del...,02126
2,"22 Bartlett St, Malden, MA 02148",750000.0,3.0,3.0,1778.0 sqft,812500.0,3499.0,5,https://www.zillow.com/homedetails/22-Bartlett...,02148
3,"9 Hall Pl, Quincy, MA 02169",750000.0,3.0,5.0,2907.0 sqft,774200.0,4300.0,3,https://www.zillow.com/homedetails/9-Hall-Pl-Q...,02169
4,"50 Myrtle St, Everett, MA 02149",750000.0,2.0,4.0,2588.0 sqft,775900.0,3499.0,5,https://www.zillow.com/homedetails/50-Myrtle-S...,02149


In [9]:
#Checking if the datasets are good
mass_data.head()

,zip_code,county_name,city_name,aland,median_household_income,median_household_income_moe,average_household_income,average_household_income_moe,per_capita_income,per_capita_income_moe,...,unemployment_pct,housing_units,median_value_of_owner_occupied_units,households,hh_families,hh_mc_families,hh_mc_with_own_children_under_18,hh_sp_families,hh_sp_with_own_children_under_18,hh_non_families
0,01001,Hampden,Agawam Town,29796888,63949,3784,81177,5434,36540,2578,...,0.039,7623,215700,7413,4164,3142,806,1022,527,3249
1,01002,Franklin; Hampshire,Amherst; Pelham; Shutesbury,142616784,61159,8382,91025,6080,30847,1871,...,0.058,10965,355500,9798,4980,3513,1171,1467,901,4818
2,01003,Hampshire,Amherst,1842387,2499,-1,-1,-1,4378,458,...,0.189,79,-1,42,4,0,0,4,0,38
3,01005,Worcester,Barre,114638390,67302,6752,95841,9282,37464,3651,...,0.029,2156,236400,1944,1243,1066,383,177,60,701
4,01007,Hampshire,Belchertown,136236046,91191,4893,111617,10414,41753,3851,...,0.042,5988,276900,5563,4075,3283,1361,792,308,1488


In [24]:
#Filter all the adresses with a certain income threshhold by zip codes
def filter_good_props(thresh):
    #Create good props dataframe
    
    rowslist = []
    dict1 = {}
    for index, rowzillow in zillow_props.iterrows():
        #print(prop, row['zip'], row['address'])
        #failsafe zip doesn't exist in the other spreadsheet
        checkzip = rowzillow['zip']
        for index1, rowmassdata in mass_data.iterrows():
            checkzipmass = rowmassdata['zip_code']
            if checkzip != checkzipmass:
                #print("cannot match zip code for: " + rowzillow['address'] + " at price: " + str(rowzillow['price']))
                continue
            else: 
                incomecheck = rowmassdata['median_household_income']
                #if does match, check median household income is below threshhold to evade
                if incomecheck < thresh:
                    print("property is undesired at " + rowzillow['address'] + ', ' + "since it is below the "  + str(thresh) + " income level")
                else:
                    print("found desired property at " + rowzillow['address'] + " at price: " + str(rowzillow['price']) + ", average median income at: " + str(incomecheck))
                    dict1.update(rowzillow)
    rowslist.append(dict1)
    good_props = pd.DataFrame(rowslist, columns=['address','price','bathrooms','bedrooms','area','zestimate','rent_zestimate','days_on_zillow','property_url','zip'])
    return good_props












found desired property at 404 Lincoln St, Lexington, MA 02421 at price: 750000.0, average median income at: 178358
property is undesired at 239-241 Delhi St, Mattapan, MA 02126, since it is below the 75000 income level
property is undesired at 22 Bartlett St, Malden, MA 02148, since it is below the 75000 income level
property is undesired at 9 Hall Pl, Quincy, MA 02169, since it is below the 75000 income level
property is undesired at 50 Myrtle St, Everett, MA 02149, since it is below the 75000 income level
property is undesired at 27 Thomas Burgin Pkwy, Quincy, MA 02169, since it is below the 75000 income level
property is undesired at 27 Thomas Burgin Pkwy, Quincy, MA 02169, since it is below the 75000 income level
found desired property at 855 Worcester St, Wellesley, MA 02481 at price: 750000.0, average median income at: 223917
property is undesired at 24 Norcross Ter, Lynn, MA 01902, since it is below the 75000 income level
found desired property at 60 John St, Reading, MA 01867 a

In [26]:
#Datetime thing
import datetime
import xlsxwriter 
current_date_and_time = datetime.datetime.now()
current_date_and_time_string = str(current_date_and_time)
date = list(current_date_and_time_string)

date[4] = "_"
date[7] = "_"
date[10] = "_"
date[13] = "_"
date[16] = "_"
date[19] = "_"
usedate = "".join(date)
extension = ".xlsx"


#Export good props to manual analysis:
filename = r"C:\Users\henry\Desktop\Google Backup\House Flipping\zillowflipping\results\goodprops" + usedate + extension
writer = pd.ExcelWriter(filename, engine='xlsxwriter')
filter_good_props(75000).to_excel(writer, index=False)
    

found desired property at 404 Lincoln St, Lexington, MA 02421 at price: 750000.0, average median income at: 178358
property is undesired at 239-241 Delhi St, Mattapan, MA 02126, since it is below the 75000 income level
property is undesired at 22 Bartlett St, Malden, MA 02148, since it is below the 75000 income level
property is undesired at 9 Hall Pl, Quincy, MA 02169, since it is below the 75000 income level
property is undesired at 50 Myrtle St, Everett, MA 02149, since it is below the 75000 income level
property is undesired at 27 Thomas Burgin Pkwy, Quincy, MA 02169, since it is below the 75000 income level
property is undesired at 27 Thomas Burgin Pkwy, Quincy, MA 02169, since it is below the 75000 income level
found desired property at 855 Worcester St, Wellesley, MA 02481 at price: 750000.0, average median income at: 223917
property is undesired at 24 Norcross Ter, Lynn, MA 01902, since it is below the 75000 income level
found desired property at 60 John St, Reading, MA 01867 a